<a href="https://colab.research.google.com/github/LOHIT9177/NLP-Projects/blob/main/Twitter_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing all the required dependencies...

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from nltk import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

Data-Extraction

In [8]:
df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')
df.head()


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [9]:
df.shape


(1599999, 6)

Data Processing..

In [10]:
#naming columns ..
column_names=["target","ids","date","flag","user","text"]
df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
df.isnull().sum()


,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [12]:
df["target"].value_counts()

,count
target,
0,800000
4,800000


In [13]:
#indicating  '4' as '1' and indiacting '0' as same
df.replace({'target':{4:1}}, inplace=True)

In [14]:
df['target'].value_counts()

,count
target,
0,800000
1,800000


1 -> Positive Tweet

0 -> Negetive Tweet

Preprocessing Text :

In [15]:
text=df['text'].tolist()
print(text[0:5])


["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds', 'my whole body feels itchy and like its on fire ', "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "]


In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
def preprocess(text):
  stemmer=PorterStemmer()
  stop_words=set(stopwords.words('english'))
  tokens=word_tokenize(str(text))
  tokens=[stemmer.stem(w.lower()) for w in tokens if  w not in stop_words and w.isalnum()]
  return " ".join(tokens)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [17]:
df["preprocessed_text"]=df["text"].apply(preprocess)
df.head()

,target,ids,date,flag,user,text,preprocessed_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http awww bummer you shoulda got da...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset ca updat facebook text might cri result ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan i dive mani time ball manag save 50 t...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad i ca see


In [ ]:
print(df["preprocessed_text"])

0          switchfoot http awww bummer you shoulda got da...
1          upset ca updat facebook text might cri result ...
2          kenichan i dive mani time ball manag save 50 t...
3                            whole bodi feel itchi like fire
4                         nationwideclass behav mad i ca see
                                 ...                        
1599995                 just woke have school best feel ever
1599996               veri cool hear old walt interview http
1599997                     are readi mojo makeov ask detail
1599998    happi 38th birthday boo alll time tupac amaru ...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: preprocessed_text, Length: 1600000, dtype: object


In [18]:

X=df['preprocessed_text'].values
Y=df['target'].values
print(X)
print(Y)

['switchfoot http awww bummer you shoulda got david carr third day d'
 'upset ca updat facebook text might cri result school today also blah'
 'kenichan i dive mani time ball manag save 50 the rest go bound' ...
 'are readi mojo makeov ask detail'
 'happi 38th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph4h']
[0 0 0 ... 1 1 1]


Spliting the data...

In [19]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [20]:
vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)
print(x_train)
print(x_test)

  (0, 433330)	0.2715512373215389
  (0, 353551)	0.35707835928495074
  (0, 189031)	0.5304407757763034
  (0, 116228)	0.3741188523037634
  (0, 238306)	0.42083340506755085
  (0, 439477)	0.4475733278061909
  (1, 165777)	1.0
  (2, 116228)	0.45279343616946033
  (2, 130845)	0.1864999047532801
  (2, 405745)	0.18429122773897044
  (2, 135678)	0.2863656716858462
  (2, 404876)	0.31623409537952896
  (2, 430329)	0.3253966532145986
  (2, 85571)	0.30906842931881373
  (2, 439824)	0.32996817658765726
  (2, 269749)	0.2383917781436081
  (2, 407543)	0.149687716372068
  (2, 182282)	0.16083915259960538
  (2, 155997)	0.1847277083665833
  (2, 285639)	0.16554194228624233
  (2, 138368)	0.19982545291868284
  (2, 291061)	0.1654481813884693
  (3, 404876)	0.29055874682035415
  (3, 163853)	0.44819435760962867
  (3, 157099)	0.2699691276483681
  :	:
  (1279996, 389020)	0.21756710376899083
  (1279996, 372059)	0.3567758343432604
  (1279996, 241360)	0.536930213801943
  (1279996, 241362)	0.552521034667777
  (1279997, 13807)	

Training the model using the LogisticRegression....

In [21]:
model=LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [22]:
# accuracy score for the Training data..
x_train_pred=model.predict(x_train)
training_data_accuracy=accuracy_score(y_train,x_train_pred)
print("training_data_accuracy : " ,training_data_accuracy)

training_data_accuracy :  0.79955234375


In [23]:
# accuracy score for the Test data..
x_test_pred=model.predict(x_test)
test_data_accuracy=accuracy_score(x_test_pred,y_test)
print("test_data_accuracy : " ,test_data_accuracy)

test_data_accuracy :  0.781896875


Model Accuracy =78.18%

In [25]:
# Example input text for prediction
x_new = "Coursera stores your credit card info and keeps charging you without your notice. They still haven't got back to me. Do NOT use this website"

# Preprocess the input text
x_new = preprocess(x_new)

# Enclose x_new within a list
x_new = [x_new]

# Transform using the vectorizer
x_new = vectorizer.transform(x_new)

# Make a prediction
prediction = model.predict(x_new)

# Output the prediction
print("Prediction:", prediction)

# Interpretation based on the prediction
if prediction[0] == 1:
    print("The sentiment is Positive.")
else:
    print("The sentiment is Negative.")

# Display test data accuracy
print("test_data_accuracy:", test_data_accuracy)


Prediction: [0]
The sentiment is Negative.
test_data_accuracy: 0.781896875
